# Project 2020-2021: University Ranking 
# Foundations of Computer Science

## Gianluca Simionato [MAT. 864045], Daniele Ceccarelli [MAT. 864239]

In [107]:
import pandas as pd
import re
import numpy as np
from datetime import datetime as dt
from difflib import SequenceMatcher
import warnings
warnings.filterwarnings("ignore")

### Data Import and Exploration

*Times Higher Education World University Ranking Dataset*

In [108]:
path = 'https://raw.githubusercontent.com/GianlucaSimionato/FoCS/main/'

In [109]:
link_times = path + 'timesData.csv'
times1 = pd.read_csv(link_times, sep=',')

times1

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598,601-800,Yeungnam University,South Korea,18.6,24.3,10.9,26.5,35.4,-,"21,958",15.3,3%,48 : 52,2016
2599,601-800,Yıldız Technical University,Turkey,14.5,14.9,7.6,19.3,44.0,-,"31,268",28.7,2%,36 : 64,2016
2600,601-800,Yokohama City University,Japan,24.0,16.1,10.2,36.4,37.9,-,"4,122",3.7,3%,NaN,2016
2601,601-800,Yokohama National University,Japan,20.1,23.3,16.0,13.5,40.4,-,"10,117",12.1,8%,28 : 72,2016


In [110]:
times1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              2603 non-null   object 
 1   university_name         2603 non-null   object 
 2   country                 2603 non-null   object 
 3   teaching                2603 non-null   float64
 4   international           2603 non-null   object 
 5   research                2603 non-null   float64
 6   citations               2603 non-null   float64
 7   income                  2603 non-null   object 
 8   total_score             2603 non-null   object 
 9   num_students            2544 non-null   object 
 10  student_staff_ratio     2544 non-null   float64
 11  international_students  2536 non-null   object 
 12  female_male_ratio       2370 non-null   object 
 13  year                    2603 non-null   int64  
dtypes: float64(4), int64(1), object(9)
memor

In [111]:
times1['year'] = pd.to_datetime(times1['year'], format='%Y')
times1['year'] = times1['year'].dt.year
times1['num_students'] = times1['num_students'].replace(',','', regex=True).astype(float)
times1['international_students'] = times1['international_students'].replace('%','', regex=True).astype(float).apply(lambda x:x/100)
times1['international'] = times1['international'].replace('-',np.nan, regex=True).astype(float)
times1['income'] = times1['income'].replace('-',np.nan, regex=True).astype(float)
times1['total_score'] = times1['total_score'].replace('-',np.nan, regex=True).astype(float)
times1['world_rank'] = times1['world_rank'].replace('=','', regex=True)

times1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              2603 non-null   object 
 1   university_name         2603 non-null   object 
 2   country                 2603 non-null   object 
 3   teaching                2603 non-null   float64
 4   international           2594 non-null   float64
 5   research                2603 non-null   float64
 6   citations               2603 non-null   float64
 7   income                  2385 non-null   float64
 8   total_score             1201 non-null   float64
 9   num_students            2544 non-null   float64
 10  student_staff_ratio     2544 non-null   float64
 11  international_students  2536 non-null   float64
 12  female_male_ratio       2370 non-null   object 
 13  year                    2603 non-null   int64  
dtypes: float64(9), int64(1), object(4)
memor

In [112]:
list_female_male = ['female_ratio','male_ratio']
times1[list_female_male] = times1['female_male_ratio'].str.split(':', 1, expand=True)

for i in list_female_male:
    times1[i] = times1[i].str.strip()
    times1[i] = times1[i].replace('-',np.nan, regex=True).apply(str)
    times1[i] = times1[i].replace('None',np.nan, regex=True).apply(str)
    times1[i] = times1[i].replace('nan',np.nan, regex=True).apply(str)
    times1[i] = times1[i].apply(float)
    

times1

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female_ratio,male_ratio
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,20152.0,8.9,0.25,NaN,2011,NaN,NaN
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,2243.0,6.9,0.27,33 : 67,2011,33.0,67.0
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,11074.0,9.0,0.33,37 : 63,2011,37.0,63.0
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,15596.0,7.8,0.22,42 : 58,2011,42.0,58.0
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,NaN,94.2,7929.0,8.4,0.27,45 : 55,2011,45.0,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598,601-800,Yeungnam University,South Korea,18.6,24.3,10.9,26.5,35.4,NaN,21958.0,15.3,0.03,48 : 52,2016,48.0,52.0
2599,601-800,Yıldız Technical University,Turkey,14.5,14.9,7.6,19.3,44.0,NaN,31268.0,28.7,0.02,36 : 64,2016,36.0,64.0
2600,601-800,Yokohama City University,Japan,24.0,16.1,10.2,36.4,37.9,NaN,4122.0,3.7,0.03,NaN,2016,NaN,NaN
2601,601-800,Yokohama National University,Japan,20.1,23.3,16.0,13.5,40.4,NaN,10117.0,12.1,0.08,28 : 72,2016,28.0,72.0


In [113]:
unique_countries_times = list(times1['country'].unique())
unique_countries_times.sort()
print('Field "country" has possible duplicates, please check: ')
i,j=0,0
simili=[]
for i in range(len (unique_countries_times)):
    for j in range(len(unique_countries_times)):
        match = SequenceMatcher(a=unique_countries_times[i], b=unique_countries_times[j]).ratio()
        if (match >0.95) and (match <1 ) :
            print(unique_countries_times[i] + '   /   ' + unique_countries_times[j])
            simili.append(unique_countries_times[j])
if len(simili)==0:
    print('No duplicates')

Field "country" has possible duplicates, please check: 
Unisted States of America   /   United States of America
United Kingdom   /   Unted Kingdom
United States of America   /   Unisted States of America
Unted Kingdom   /   United Kingdom


In [114]:
times1['country'] = times1['country'].replace('Unisted States of America','United States of America', regex=True)
times1['country'] = times1['country'].replace('Unted Kingdom','United Kingdom', regex=True)

In [115]:
unique_universities_times = list(times1['university_name'].unique())
unique_universities_times.sort()
print('Field "university_name" has possible duplicates, please check: ')
i,j=0,0
simili=[]
for i in range(len(unique_universities_times)):
    for j in range(len(unique_universities_times)):
        match = SequenceMatcher(a=unique_universities_times[i], b=unique_universities_times[j]).ratio()
        if (match >0.95) and (match <1 ) :
            print(unique_universities_times[i] + '   /   ' +unique_universities_times[j])
            simili.append(unique_universities_times[j])
if len(simili)==0:
    print('No duplicates')

Field "university_name" has possible duplicates, please check: 
Iran University of Science and Technology   /   Jordan University of Science and Technology
Jordan University of Science and Technology   /   Iran University of Science and Technology
Northeastern University   /   Northwestern University
Northwestern University   /   Northeastern University
University of Freiburg   /   University of Fribourg
University of Fribourg   /   University of Freiburg


Duble checking we can state that names of abovementioned universities in Times dataset are similar but refers to different universities.  

*Academic Ranking of World Universities Dataset*

In [116]:
link_shanghai = path + 'shanghaiData.csv'
Shanghai = pd.read_csv(link_shanghai, sep=',')

Shanghai

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005
4,5,Massachusetts Institute of Technology (MIT),4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005
...,...,...,...,...,...,...,...,...,...,...,...
4892,401-500,University of Trieste,11-20,NaN,0.0,0.0,5.0,10.9,25.1,20.1,2015
4893,401-500,University of Zaragoza,9-13,NaN,0.0,0.0,7.6,5.1,33.3,13.1,2015
4894,401-500,Utah State University,126-146,NaN,13.6,0.0,3.6,10.8,25.1,15.5,2015
4895,401-500,Vienna University of Technology,4-6,NaN,0.0,0.0,0.0,12.2,28.8,22.9,2015


In [117]:
Shanghai= Shanghai[Shanghai.university_name==Shanghai.university_name]

In [118]:
unique_universities_Shanghai = list(Shanghai['university_name'].unique())
print('Field "university_name" has possible duplicates, please check: ')
i,j=0,0
simili=[]
for i in range(len(unique_universities_Shanghai)):
    for j in range(len(unique_universities_Shanghai)):
        match = SequenceMatcher(a=unique_universities_Shanghai[i], b=unique_universities_Shanghai[j]).ratio()
        if (match >0.95) and (match <1 ) :
            print(unique_universities_Shanghai[i] + '   /   ' + unique_universities_Shanghai[j])
            simili.append(unique_universities_Shanghai[j])
if len(simili)==0:
    print('No duplicates')

Field "university_name" has possible duplicates, please check: 
University of California, Berkeley   /   University of California-Berkeley
University of Michigan - Ann Arbor   /   University of Michigan-Ann Arbor
Northwestern University   /   Northeastern University
University of Paris Sud (Paris 11)   /   University of Paris-Sud (Paris 11)
University of Freiburg   /   University of Fribourg
Swiss Federal Institute of Technology of Lausanne   /   Swiss Federal Institute of Technology Lausanne
University of Bordeaux 1   /   University of Bordeaux
University of Genova   /   University of Genoa
University of Montpellier 2   /   University of Montpellier
London School of Hygiene and Tropical Medicine   /   London School of Hygiene & Tropical Medicine
Northeastern University   /   Northwestern University
University of Fribourg   /   University of Freiburg
University of Sevilla   /   University of Seville
Aix-Marseille University   /   Aix Marseille University
Aix Marseille University   /   

We see that there may be different mispelling in university names, based on our knowledge we feel free to merge the following names.

In [119]:
Shanghai['university_name'] = Shanghai['university_name'].replace('University of California, Berkeley','University of California-Berkeley', regex=True)
Shanghai['university_name'] = Shanghai['university_name'].replace('University of Michigan - Ann Arbor','University of Michigan-Ann Arbor', regex=True)
Shanghai['university_name'] = Shanghai['university_name'].replace('University of Paris Sud (Paris 11)','University of Paris-Sud (Paris 11)', regex=True)
Shanghai['university_name'] = Shanghai['university_name'].replace('University of Genova','University of Genova', regex=True)
Shanghai['university_name'] = Shanghai['university_name'].replace('Swiss Federal Institute of Technology Lausanne','Swiss Federal Institute of Technology of Lausanne', regex=True)
Shanghai['university_name'] = Shanghai['university_name'].replace('London School of Hygiene and Tropical Medicine','London School of Hygiene & Tropical Medicine', regex=True)
Shanghai['university_name'] = Shanghai['university_name'].replace('University of Sevilla','University of Seville', regex=True)
Shanghai['university_name'] = Shanghai['university_name'].replace('Texas A & M University','Texas A&M University', regex=True)
Shanghai['university_name'] = Shanghai['university_name'].replace('Queen Mary, University of London','Queen Mary University of London', regex=True)
Shanghai['university_name'] = Shanghai['university_name'].replace('University of Pittsburgh, Pittsburgh Campus','University of Pittsburgh-Pittsburgh Campus', regex=True)
Shanghai['university_name'] = Shanghai['university_name'].replace('Aix Marseille University','Aix-Marseille University', regex=True)

In [120]:
link_school_and_country = path + 'school_and_country_table.csv'
school_and_country = pd.read_csv(link_school_and_country, sep=',')

school_and_country

,school_name,country
0,Harvard University,United States of America
1,California Institute of Technology,United States of America
2,Massachusetts Institute of Technology,United States of America
3,Stanford University,United States of America
4,Princeton University,United States of America
...,...,...
813,Xidian University,China
814,Yeungnam University,South Korea
815,Yıldız Technical University,Turkey
816,Yokohama City University,Japan


In [121]:
Shanghai = pd.merge(Shanghai, school_and_country, 
                left_on='university_name',
                right_on='school_name',
                how='left')
Shanghai.drop('school_name', axis=1, inplace=True)
Shanghai

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,country
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005,United States of America
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005,United Kingdom
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005,United States of America
3,4,University of California-Berkeley,3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005,NaN
4,5,Massachusetts Institute of Technology (MIT),4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4891,401-500,University of Trieste,11-20,NaN,0.0,0.0,5.0,10.9,25.1,20.1,2015,Italy
4892,401-500,University of Zaragoza,9-13,NaN,0.0,0.0,7.6,5.1,33.3,13.1,2015,Spain
4893,401-500,Utah State University,126-146,NaN,13.6,0.0,3.6,10.8,25.1,15.5,2015,NaN
4894,401-500,Vienna University of Technology,4-6,NaN,0.0,0.0,0.0,12.2,28.8,22.9,2015,Austria


In [122]:
print('The following universities are not mapped in \'school_and_country\' dataset: ')
set(list(Shanghai[Shanghai['country'].isna()]['university_name']))

The following universities are not mapped in 'school_and_country' dataset: 


{'Arizona State University - Tempe',
 'Baylor College of Medicine',
 'Beihang University',
 'Beijing Normal University',
 'Beijing University of Aeronautics and Astronautics',
 'Brigham Young University',
 'Brunel University',
 'Capital University of Medical Sciences',
 'Catholic University of Chile',
 'Catholic University of Korea',
 'Catholic University of Leuven',
 'Catholic University of Louvain',
 'Central South University',
 'China Medical University',
 'City University of New York City College',
 'College of France',
 'Cranfield University',
 'Curtin University of Technology',
 'Dresden University of Technology',
 'ESPCI ParisTech',
 'Ecole National Superieure Mines - Paris',
 'Ecole Normale Superieure - Lyon',
 'Ecole Normale Superieure - Paris',
 'Ecole Polytechnique',
 'Eotvos Lorand University',
 'Erasmus University',
 'Federal University of Sao Paulo',
 'Gunma University',
 'Hannover Medical School',
 'Henri Poincare University (Nancy 1)',
 'Icahn School of Medicine at Moun

In [123]:
Shanghai.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4896 entries, 0 to 4895
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   world_rank       4896 non-null   object 
 1   university_name  4896 non-null   object 
 2   national_rank    4896 non-null   object 
 3   total_score      1101 non-null   float64
 4   alumni           4896 non-null   float64
 5   award            4895 non-null   float64
 6   hici             4895 non-null   float64
 7   ns               4875 non-null   float64
 8   pub              4895 non-null   float64
 9   pcp              4895 non-null   float64
 10  year             4896 non-null   int64  
 11  country          3014 non-null   object 
dtypes: float64(7), int64(1), object(4)
memory usage: 497.2+ KB


*Center for World University Rankings Dataset* 

In [124]:
link_cwur = path + 'cwurData.csv'
Cwur = pd.read_csv(link_cwur, sep=',')
Cwur.rename(columns={'institution':'university_name'},inplace=True)

Cwur

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,996,University of the Algarve,Portugal,7,367,567,218,926,845,812,969.0,816,44.03,2015
2196,997,Alexandria University,Egypt,4,236,566,218,997,908,645,981.0,871,44.03,2015
2197,998,Federal University of Ceará,Brazil,18,367,549,218,830,823,812,975.0,824,44.03,2015
2198,999,University of A Coruña,Spain,40,367,567,218,886,974,812,975.0,651,44.02,2015


In [125]:
unique_countries_times = list(Cwur['country'].unique())
unique_countries_times.sort()

print('Field "country" has possible duplicates, please check: ')
simili = []
i,j=0,0
for i in range(len (unique_countries_times)):
    for j in range(len(unique_countries_times)):
        match = SequenceMatcher(a=unique_countries_times[i], b=unique_countries_times[j]).ratio()
        if (match >0.95) and (match <1 ) :
            print(unique_countries_times[i] + '   /   ' + unique_countries_times[j])
            simili.append(unique_countries_times[j])
if len(simili)==0:
    print('No duplicates')

Field "country" has possible duplicates, please check: 
No duplicates


In [126]:
unique_universities_Cwur = list(Cwur['university_name'].unique())
print('Field "university_name" has possible duplicates, please check: ')
i,j=0,0
simili=[]
for i in range(len(unique_universities_Cwur)):
    for j in range(len(unique_universities_Cwur)):
        match = SequenceMatcher(a=unique_universities_Cwur[i], b=unique_universities_Cwur[j]).ratio()
        if (match >0.95) and (match <1 ) :
            print(unique_universities_Cwur[i] + '   /   ' + unique_universities_Cwur[j])
            simili.append(unique_universities_Cwur[j])
if len(simili)==0:
    print('No duplicates')

Field "university_name" has possible duplicates, please check: 
Northwestern University   /   Northeastern University
University of Montpellier 2   /   University of Montpellier 1
University of Montpellier 2   /   University of Montpellier
Northeastern University   /   Northwestern University
University of Bordeaux II   /   University of Bordeaux I
University of Bordeaux I   /   University of Bordeaux II
University of Bordeaux I   /   University of Bordeaux
University of Montpellier 1   /   University of Montpellier 2
University of Montpellier 1   /   University of Montpellier
University of Salerno   /   University of Salento
University of Salento   /   University of Salerno
Hunan Normal University   /   Henan Normal University
University of Montpellier   /   University of Montpellier 2
University of Montpellier   /   University of Montpellier 1
University of Bordeaux   /   University of Bordeaux I
Henan Normal University   /   Hunan Normal University


We can see that there are universities with very similar names, in particular Montpellier and Bordeaux has universities differentiated only by a number, double checking on the web we can state that actually refer to different universities. 

In [127]:
Cwur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   world_rank            2200 non-null   int64  
 1   university_name       2200 non-null   object 
 2   country               2200 non-null   object 
 3   national_rank         2200 non-null   int64  
 4   quality_of_education  2200 non-null   int64  
 5   alumni_employment     2200 non-null   int64  
 6   quality_of_faculty    2200 non-null   int64  
 7   publications          2200 non-null   int64  
 8   influence             2200 non-null   int64  
 9   citations             2200 non-null   int64  
 10  broad_impact          2000 non-null   float64
 11  patents               2200 non-null   int64  
 12  score                 2200 non-null   float64
 13  year                  2200 non-null   int64  
dtypes: float64(2), int64(10), object(2)
memory usage: 240.8+ KB


## 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [128]:
times_max = times1.iloc[times1.groupby('university_name')['year'].idxmax()]
times_max

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female_ratio,male_ratio
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,NaN,35569.0,17.0,0.01,-,2016,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,0.15,48 : 52,2016,48.0,52.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,0.17,32 : 68,2016,32.0,68.0
1908,106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,0.14,54 : 46,2016,54.0,46.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,0.18,48 : 52,2016,48.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,2400.0,7.9,0.20,46 : 54,2016,46.0,54.0
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,NaN,2218.0,8.0,0.14,49 : 51,2016,49.0,51.0
1904,101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,2429.0,4.8,0.30,18 : 82,2016,18.0,82.0
1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,9666.0,10.5,0.54,27 : 73,2016,27.0,73.0


In [129]:
times_min = times1.iloc[times1.groupby('university_name')['year'].idxmin()]
times_min

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female_ratio,male_ratio
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,NaN,35569.0,17.0,0.01,-,2016,NaN,NaN
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,NaN,17422.0,15.9,0.15,48 : 52,2012,48.0,52.0
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,NaN,16099.0,24.2,0.17,32 : 68,2012,32.0,68.0
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,23895.0,13.6,0.14,54 : 46,2011,54.0,46.0
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,NaN,9252.0,19.2,0.18,48 : 52,2012,48.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,42,École Normale Supérieure,France,66.8,44.9,48.2,95.7,30.7,68.6,2400.0,7.9,0.20,46 : 54,2011,46.0,54.0
99,100,École Normale Supérieure de Lyon,France,51.1,37.6,34.4,88.8,26.1,57.0,2218.0,8.0,0.14,49 : 51,2011,49.0,51.0
38,39,École Polytechnique,France,57.9,77.9,56.1,91.4,NaN,69.5,2429.0,4.8,0.30,18 : 82,2011,18.0,82.0
47,48,École Polytechnique Fédérale de Lausanne,Switzerland,55.0,100.0,56.1,83.8,38.0,66.5,9666.0,10.5,0.54,27 : 73,2011,27.0,73.0


## 2. For each university, compute the improvement in income between the least recent and the most recent data points

In [130]:
times_max1 = times_max[['university_name','year','income']]
times_min1 = times_min[['university_name','year','income']]

times_merge = pd.merge(times_max1, times_min1, 
                       on='university_name',
                       suffixes =('_max', '_min'))

times_merge['income_difference_absolute'] = times_merge['income_max'] - times_merge['income_min']
times_merge['income_difference_relative'] = times_merge['income_max'] / times_merge['income_min']-1
times_merge['income_difference_relative%'] = times_merge['income_difference_relative']*100
times_merge.set_index(['university_name'], inplace= True)

times_merge

,year_max,income_max,year_min,income_min,income_difference_absolute,income_difference_relative,income_difference_relative%
university_name,,,,,,,
AGH University of Science and Technology,2016,NaN,2016,NaN,NaN,NaN,NaN
Aalborg University,2016,43.7,2012,36.4,7.3,0.200549,20.054945
Aalto University,2016,61.6,2012,61.9,-0.3,-0.004847,-0.484653
Aarhus University,2016,68.3,2011,61.5,6.8,0.110569,11.056911
Aberystwyth University,2016,31.3,2012,35.5,-4.2,-0.118310,-11.830986
...,...,...,...,...,...,...,...
École Normale Supérieure,2016,37.1,2011,30.7,6.4,0.208469,20.846906
École Normale Supérieure de Lyon,2016,31.7,2011,26.1,5.6,0.214559,21.455939
École Polytechnique,2016,82.3,2011,NaN,NaN,NaN,NaN


## 3. Find the university with the largest increase computed in the previous point

Let's verify that the university with the largest abslute increase matches the largest increase in relative terms

In [131]:
times_merge.income_difference_relative.idxmax() == times_merge.income_difference_absolute.idxmax()

True

In [132]:
print('The university with the largest increase is:\n' + times_merge.income_difference_absolute.idxmax()+\
      '\nwith an absolute value of ' + str(round(times_merge.loc[times_merge.income_difference_absolute.idxmax()]['income_difference_absolute'],2))+\
      ' that represents the ' + str(int(times_merge.loc[times_merge.income_difference_relative.idxmax()]['income_difference_relative%'])*100) +'%'  )

The university with the largest increase is:
TU Dresden
with an absolute value of 67.8 that represents the 21200%


## 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

In [133]:
times_max = times1.iloc[times1.groupby('university_name').year.idxmax()]
Shanghai_max = Shanghai.iloc[Shanghai.groupby('university_name').year.idxmax()]
Cwur_max = Cwur.iloc[Cwur.groupby('university_name').year.idxmax()]

We then use a function to make the field "world_rank" numeric 

In [134]:
def World_Rank_Adjust(df_max):
    
    df_max['world_rank'] = df_max['world_rank'].str.replace(r'^=', '')

    list10 = ['lower_bound','upper_bound']

    df_max[list10] = df_max['world_rank'].str.split('-', expand=True)

    df_max['lower_bound'].replace(to_replace=[None],  value=0, inplace=True)
    df_max['upper_bound'].replace(to_replace=[None],  value=0, inplace=True)
    
    df_max['lower_bound'] = df_max['lower_bound'].apply(float)
    df_max['upper_bound'] = df_max['upper_bound'].apply(float)

    df_max['simple_mean'] = (df_max['upper_bound']-df_max['lower_bound'])//2 + df_max['lower_bound']

    df_max['world_rank_Adjusted'] = np.where(df_max['world_rank'].str.contains("-"),
                                       df_max['simple_mean'],
                                       df_max['world_rank'])
    
    df_max.drop(columns=list10, inplace= True)
    df_max.drop(columns='simple_mean', inplace= True)
    df_max['world_rank_Adjusted'] = df_max['world_rank_Adjusted'].apply(int)
    df_max.sort_values(by=['world_rank_Adjusted'])
    return df_max

In [135]:
times_max = World_Rank_Adjust(times_max)
Shanghai_max = World_Rank_Adjust(Shanghai_max)
Cwur_max['world_rank_Adjusted'] = Cwur_max['world_rank']


In [136]:
df_Ranking_Comparison = pd.merge(times_max[['university_name','world_rank_Adjusted']],
                        Shanghai_max[['university_name','world_rank_Adjusted']],
                        how='outer', 
                        on='university_name',
                        suffixes=('_Times','_Shanghai'))

df_Ranking_Comparison = pd.merge(df_Ranking_Comparison,
                        Cwur_max[['university_name','world_rank_Adjusted']],
                        how='outer', 
                        on='university_name')
                        
df_Ranking_Comparison.rename(columns={'world_rank_Adjusted':'world_rank_Adjusted_Cwur'},inplace=True)
df_Ranking_Comparison.set_index(['university_name'], inplace= True)
df_Ranking_Comparison.dropna(inplace=True)
df_Ranking_Comparison['MAX'] = df_Ranking_Comparison[['world_rank_Adjusted_Cwur', 'world_rank_Adjusted_Shanghai', 'world_rank_Adjusted_Times']].max(axis=1)
df_Ranking_Comparison['min'] = df_Ranking_Comparison[['world_rank_Adjusted_Cwur', 'world_rank_Adjusted_Shanghai', 'world_rank_Adjusted_Times']].min(axis=1)
df_Ranking_Comparison['diff'] =df_Ranking_Comparison['MAX']-df_Ranking_Comparison['min']

Let's verify that the values for the Aarhus University match the indications

In [137]:
df_Ranking_Comparison.loc['Aarhus University']

world_rank_Adjusted_Times       106.0
world_rank_Adjusted_Shanghai     73.0
world_rank_Adjusted_Cwur        122.0
MAX                             122.0
min                              73.0
diff                             49.0
Name: Aarhus University, dtype: float64

In [138]:
print('L\' The university with the largest difference is: \n' + df_Ranking_Comparison['diff'].idxmax())

df_Ranking_Comparison.loc[df_Ranking_Comparison['diff'].idxmax()]

L' The university with the largest difference is: 
King Abdulaziz University


world_rank_Adjusted_Times       275.0
world_rank_Adjusted_Shanghai    175.0
world_rank_Adjusted_Cwur        995.0
MAX                             995.0
min                             175.0
diff                            820.0
Name: King Abdulaziz University, dtype: float64

## 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [139]:
times2 = times1.copy()
times_countries_max_year = times2.iloc[times2.groupby('university_name').year.idxmax()]
times_countries_max_year = times_countries_max_year[times_countries_max_year['female_male_ratio'].notna()]
times_countries_max_year['total_females'] = times_countries_max_year['num_students'] * times_countries_max_year['female_ratio']
times_countries_max_year['total_males'] = times_countries_max_year['num_students'] * times_countries_max_year['male_ratio']
times_countries_max_year_sum = times_countries_max_year.groupby('country')['total_females','total_males'].sum()
times_countries_max_year_sum1 = pd.DataFrame(times_countries_max_year_sum)

times_countries_max_year_sum1

,total_females,total_males
country,,
Argentina,6719126.0,4118174.0
Australia,39173588.0,32164012.0
Austria,6836408.0,6611292.0
Bangladesh,2132344.0,4139256.0
Belarus,2021907.0,908393.0
...,...,...
Uganda,1867000.0,1867000.0
Ukraine,1784600.0,1925000.0
United Arab Emirates,951644.0,493056.0


## 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [140]:
times2['female_male_ratio2'] = times2['female_ratio'] / times2['male_ratio']
times2['female_male_ratio2'].unique()
times2['female_male_ratio2'] = times2['female_male_ratio2'].replace(np.inf,1)
times2_new = times2.groupby(['university_name'])['university_name','female_male_ratio2'].mean()
times2_new1 = pd.DataFrame(times2_new)
times2_new1.reset_index(inplace=True)
average_female_male_ratio = times2_new1['female_male_ratio2'].mean()
print('the average male female ratio is: '+ str(round(average_female_male_ratio,2)))
print('Number of universities below average:',sum(times2_new1['female_male_ratio2'] < average_female_male_ratio))
times2_new_below_average = times2_new[times2_new1['female_male_ratio2'] < average_female_male_ratio]
times2_new_below_average.set_index('university_name',inplace=True)
times2_new_below_average.rename(columns={'female_male_ratio2':'female_male_ratio'},inplace=True)

times2_new_below_average

the average male female ratio is: 1.08
Number of universities below average: 347


,female_male_ratio
university_name,
Aalborg University,0.923077
Aalto University,0.470588
Aberystwyth University,0.923077
Ajou University,0.492537
Alexandria University,0.851852
...,...
Zhejiang University,0.694915
École Normale Supérieure,0.851852
École Normale Supérieure de Lyon,0.960784


## 7. For each country compute the fraction of students that are in one of the universities computed in the previous point.

In [141]:
times7 = times2.groupby(['country'])['num_students'].sum()
times7 = pd.DataFrame(times7)
times7.reset_index(inplace=True)
times7.rename(columns={'num_students':'num_students_COUNTRY'}, inplace=True)

times8 = times2.groupby(['university_name'])['num_students'].sum()
times8 = pd.DataFrame(times8)
times8.reset_index(inplace=True)
times8.rename(columns={'num_students':'num_students_UNI'}, inplace=True)

df_UNI_below = pd.merge(times2_new_below_average,
                        times8,
                        how='left', 
                        left_index=True,
                        right_on='university_name')

df_UNI_below.drop(columns='female_male_ratio', inplace=True)
df_mapping = times2[['university_name','country']]
df_mapping.drop_duplicates(inplace= True)

df_UNI_below = pd.merge(df_UNI_below,
                        df_mapping,
                        how='left', 
                        on='university_name')

df_UNI_below1 = df_UNI_below.groupby(['country'])['num_students_UNI'].sum()
df_UNI_below1 = pd.DataFrame(df_UNI_below1)
df_UNI_below1.reset_index(inplace=True)

df_UNI_below = pd.merge(df_UNI_below1,
                        times7,
                        how='left', 
                        on='country')

df_UNI_below['rate']= df_UNI_below.num_students_UNI / df_UNI_below.num_students_COUNTRY
df_UNI_below = df_UNI_below[['country','rate' ]]
df_UNI_below.set_index('country',inplace=True)

df_UNI_below

,rate
country,
Australia,0.211529
Austria,0.371142
Bangladesh,1.000000
Brazil,0.735811
Canada,0.091894
Chile,0.671834
China,0.713400
Colombia,1.000000
Czech Republic,0.094567


## 8. Read the file educational_attainment_supplementary_data.csv, discarding any row without country_name or series_name

In [142]:
link_educational_attainment_supplementary_data = path + 'educational_attainment_supplementary_data.csv'
edu_att = pd.read_csv(link_educational_attainment_supplementary_data, sep=',')
subset1 = ['country_name', 'series_name']
edu_att.dropna(axis=0, subset=subset1, inplace = True)
edu_att2 = edu_att.copy()
edu_att.set_index(['country_name', 'series_name'], inplace=True)

edu_att

1985  1986  \
country_name series_name                                                      
Afghanistan  Barro-Lee: Average years of primary schooling, ...  0.33   NaN   
             Barro-Lee: Average years of primary schooling, ...  1.03   NaN   
             Barro-Lee: Average years of primary schooling, ...  0.83   NaN   
             Barro-Lee: Average years of primary schooling, ...  2.34   NaN   
             Barro-Lee: Average years of primary schooling, ...  0.54   NaN   
...                                                               ...   ...   
Zimbabwe     UIS: Percentage of population age 25+ with some...   NaN   NaN   
             UIS: Percentage of population age 25+ with some...   NaN   NaN   
             UIS: Percentage of population age 25+ with unkn...   NaN   NaN   
             UIS: Percentage of population age 25+ with unkn...   NaN   NaN   
             UIS: Percentage of population age 25+ with unkn...   NaN   NaN   

                                                                 1987  1990  \
country_name series_name                                                      
Afghanistan  Barro-Lee: Average years of primary schooling, ...   NaN  0.44   
             Barro-Lee: Average years of primary schooling, ...   NaN  1.26   
             Barro-Lee: Average years of primary schooling, ...   NaN  0.95   
             Barro-Lee: Average years of primary schooling, ...   NaN  2.22   
             Barro-Lee: Average years of primary schooling, ...   NaN  0.92   
...                                                               ...   ...   
Zimbabwe     UIS: Percentage of population age 25+ with some...   NaN   NaN   
             UIS: Percentage of population age 25+ with some...   NaN   NaN   
             UIS: Percentage of population age 25+ with unkn...   NaN   NaN   
             UIS: Percentage of population age 25+ with unkn...   NaN   NaN   
             UIS: Percentage of population age 25+ with unkn...   NaN   NaN   

                                                                 1991  1992  \
country_name series_name                                                      
Afghanistan  Barro-Lee: Average years of primary schooling, ...   NaN   NaN   
             Barro-Lee: Average years of primary schooling, ...   NaN   NaN   
             Barro-Lee: Average years of primary schooling, ...   NaN   NaN   
             Barro-Lee: Average years of primary schooling, ...   NaN   NaN   
             Barro-Lee: Average years of primary schooling, ...   NaN   NaN   
...                                                               ...   ...   
Zimbabwe     UIS: Percentage of population age 25+ with some...   NaN   NaN   
             UIS: Percentage of population age 25+ with some...   NaN   NaN   
             UIS: Percentage of population age 25+ with unkn...   NaN   NaN   
             UIS: Percentage of population age 25+ with unkn...   NaN   NaN   
             UIS: Percentage of population age 25+ with unkn...   NaN   NaN   

                                                                 1993  1995  \
country_name series_name                                                      
Afghanistan  Barro-Lee: Average years of primary schooling, ...   NaN  0.57   
             Barro-Lee: Average years of primary schooling, ...   NaN  1.54   
             Barro-Lee: Average years of primary schooling, ...   NaN  1.26   
             Barro-Lee: Average years of primary schooling, ...   NaN  2.37   
             Barro-Lee: Average years of primary schooling, ...   NaN  0.94   
...                                                               ...   ...   
Zimbabwe     UIS: Percentage of population age 25+ with some...   NaN   NaN   
             UIS: Percentage of population age 25+ with some...   NaN   NaN   
             UIS: Percentage of population age 25+ with unkn...   NaN   NaN   
             UIS: Percentage of population age 25+ with unkn...   NaN   NaN   
             UIS: Percentage of p

## 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [143]:
col_value_vars =list(edu_att2.columns)[2:]
col_id_vars =list(edu_att2.columns)[:2]
edu_att9 = pd.melt( edu_att2,
                    id_vars = col_id_vars,
                    value_vars = col_value_vars,
                    var_name='year', value_name='value')

edu_att9

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54
...,...,...,...,...
2134345,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134346,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134347,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN
2134348,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN


In [144]:
print('The number of rows in the “melted” data frame is equal to the product of the rows and columns of edu_att dataframe (multindex)')
print(str(edu_att.shape[0]*edu_att.shape[1]))

The number of rows in the “melted” data frame is equal to the product of the rows and columns of edu_att dataframe (multindex)
2134350


## 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [145]:
frames = [times1, Shanghai, Cwur]
rankings_concat = pd.concat(frames)
rankings_concat1 = rankings_concat.groupby(['university_name'])['world_rank'].count()
rankings_concat1 = pd.DataFrame(rankings_concat1)
rankings_concat1 = rankings_concat1.rename(columns={'world_rank': 'Count_ranking'})

rankings_concat1

,Count_ranking
university_name,
AGH University of Science and Technology,3
Aalborg University,9
Aalto University,14
Aarhus University,19
Aberystwyth University,7
...,...
École centrale de Lyon,1
École normale supérieure - Paris,4
École normale supérieure de Cachan,1


## 11. In the times ranking, compute the number of times each university appears

In [146]:
times11 = times1.groupby(['university_name'])['world_rank'].count()
times11 = pd.DataFrame(times11)
times11 = times11.rename(columns={'world_rank': 'Count_ranking_Times'})

times11

,Count_ranking_Times
university_name,
AGH University of Science and Technology,1
Aalborg University,5
Aalto University,5
Aarhus University,6
Aberystwyth University,5
...,...
École Normale Supérieure,6
École Normale Supérieure de Lyon,6
École Polytechnique,6


## 12. Find the universities that appear at most twice in the times ranking.

In [147]:
times12 = times11[times11['Count_ranking_Times'] <= 2]

times12

,Count_ranking_Times
university_name,
AGH University of Science and Technology,1
Adam Mickiewicz University,1
Aix-Marseille University,1
Ajou University,1
Alexandru Ioan Cuza University,1
...,...
Yokohama City University,1
Yokohama National University,1
Yuan Ze University,2


## 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [148]:
rankings_merge = pd.merge(times1,
                          Shanghai, 
                          on=['world_rank','university_name','year'],
                          how='inner')


Cwur['world_rank']= Cwur['world_rank'].astype(str)

rankings_merge = pd.merge(rankings_merge,
                          Cwur, 
                          on=['world_rank','university_name','year'],
                          how='inner')

rankings_merge.set_index('university_name',inplace=True)

rankings_merge = rankings_merge[['year', 'world_rank']]

rankings_merge

,year,world_rank
university_name,,
Stanford University,2013,2
